In [2]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET
import math
import scipy.signal as signal
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2 as cv

ModuleNotFoundError: No module named 'cv2'

In [ ]:
### 加载信号数据
def load_signal(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        data = list(float(x) for x in filter(None,f.read().split("\t"))) ##过滤空值,按列存储；f.read()一次性读取文件有内存溢出风险
        
    return data

In [ ]:
### 信号读取并预处理--开关标准化（0，1），功率信号中值滤波去噪
def load_data(path,method_denoise="medfilt",INDEX=0): 
    list_ = os.listdir(path)
    data11 = load_signal(path + "\\" + list_[INDEX * 3 + 1])  #INDEX考虑激光器索引 默认list_[0]为xml路径文件
    data12 = load_signal(path + "\\" + list_[INDEX * 3 + 2])
    data13 = load_signal(path + "\\" + list_[INDEX * 3 + 3])
    data12_nor = np.where(np.array(data12) > 2500, 1, 0)  # 开关数据处理为0，1数据
    method_denoises = {"medfilt": denoise.medfilt,  ###降噪算法 #中值滤波
                       "MedianAverage": denoise.MedianAverage,  #中位值平均滤波降噪
                       "SlidingAverage": denoise.SlidingAverage,  #移动平均滤波降噪
                       "wave_denoise": denoise.wave_denoise,  # 小波去噪
                       "wave_denoise1": denoise.wave_denoise1,  #小波近似层
                       "Fourier_denoise":denoise.Fourier_denoise,
                       "usv_denoise":denoise.usv_denoise,
                       "AmplitudeLimitingAverage":denoise.AmplitudeLimitingAverage}
    data_13_nonoise=method_denoises[method_denoise](data13) #功率信号中中值滤波
    data11 = method_denoises[method_denoise](data11)   #强度信号中值滤波
    if(data_13_nonoise[-1] != 0):
        data_13_nonoise[-1] = 0                       #功率非低电平结尾处理
    print(len(data11), len(data12_nor), len(data_13_nonoise))  #数据长度信息
    return list_, data11, data12_nor, data_13_nonoise

In [ ]:
#####开关上升索引及下降索引#####
def switch_index(data12_nor):
    cur_up_index= list(np.where(np.diff(data12_nor) > 0)[0])
    cur_down_index = list(np.where(np.diff(data12_nor) < 0)[0])
    if data12_nor[len(data12_nor) - 1] == 1:  
        cur_down_index.append(len(data12_nor) - 1)   #以上升沿结尾的处理？？？
    ##判断上升和下降起始位置顺序
    if len(cur_up_index) == 0 or len(cur_down_index) == 0:
        cur_up_index=[]
        cur_down_index= []
    else:
        if cur_up_index[0] > cur_down_index[0]:
            cur_down_index = cur_down_index[1:]  #如果上升沿起始索引大于下降沿，下降沿索引后移一位？？？ 
    return cur_up_index, cur_down_index


In [ ]:
#####合并列表2023.1.6修改#####  功率相同时合并，pandas应该有内置算法。
def concat_func(row):
    return pd.Series({
        'speed':np.append((row['speed'].values)[0],(row['speed'].values)[1]), ##int型
        'laser_index': int(row['laser_index'].unique()), #np.append((row['laser_index'].values)[0],(row['laser_index'].values)[1]),  ##拼接
        'count':  np.append((row['count'].values)[0],(row['count'].values)[1]),
        'type':  np.append((row['type'].values)[0],(row['type'].values)[1]),
        'polyline': (row['polyline'].values)[0]+(row['polyline'].values)[1]
    })
def concat_adjacent_power(data_feature):
    ##2022.11.29power相等合并特征
    drop_index = []
    for i in range(len(data_feature)-1):
        if data_feature['power'][i]==data_feature['power'][i+1]:
            data_feature[i:i + 2]=data_feature[i:i + 2].groupby(data_feature['power']).apply(
                lambda row: concat_func(row)).reset_index()  # 合并相邻两个power相等的行
            drop_index.append(i)
    data_feature=data_feature.drop(labels=drop_index,axis=0)
    return data_feature

In [ ]:
#解析xml，并将扫描策略排序
def extract_path_xml(xml_path, INDEX=0):
    tree = ET.parse(xml_path)
    feature = tree.findall("FEATURE")
    data_feature_original = pd.DataFrame()  ##创建空表
    for i, child_f in enumerate(feature):
        index_ = int(child_f.get('INDEX')) #获取feature的几个属性index=0,1代表不同激光器
        type_ = int(child_f.get('TYPE')) # 不同特征，5为实体
        count_ = int(child_f.get('COUNT')) #count=1，0，代表扫描几次
        power_ = int(child_f.get('POWER')) #功率
        speed_ = int(child_f.get('SPEED')) #速度
        
        
        ##扫描策略排序
        type_i_line_point = []
        for i, child_p in enumerate(child_f.iterfind("POLYLINE")):
            polyline_array = np.array(child_p.text.split(',')).astype(float)
            type_i_line_point.append(list(polyline_array[1:]))  # 第一位是点数 n，第二位开始是坐标
        
        ##提取有效激光扫描的特征
        if index_ ==INDEX:
            if power_ != 0 and count_ !=0:
                data_feature_original = pd.concat([data_feature_original,
                                                  pd.DataFrame({'power': [power_], 'speed': [speed_], 'laser_index': [index_],
                                                 'count': [count_], 'type': [type_], 'polyline': [type_i_line_point]})],
                                                  ignore_index =True)
    ## 复制合并前列表
    data_feature = data_feature_original[['power', 'speed', 'laser_index', 'count', 'type']]
    data_feature['polyline'] = data_feature_original.polyline.str.len() ##用polyline长度代替坐标点，相关计算节省开销 此处优化空间。
    
    ## 功率相等合并特征
    data_feature = concat_adjacent_power(data_feature)
    _index = data_feature.index.values  # 特征行索引 并未输出此值，何用？？？？
    return data_feature, data_feature_original

In [15]:
xml_path = r'C:\jupyter notebook\熔池数据\20210316_20210316094436_K438\layer100\Data\100_20210316121607.xml'
tree = ET.parse(xml_path)
# root = tree.getroot() #获取根节点，数据处理未用上
feature = tree.findall('FEATURE') #获取feature节点列表

all_type_line_point = []
num_poly = []
data_feature = pd.DataFrame()

#获取feature列表的特征属性信息
for i, child_f in enumerate(feature):
    index_ = int(child_f.get('INDEX')) #获取feature的几个属性index=0,1代表不同激光器
    type_ = int(child_f.get('TYPE')) # 不同特征，5为实体
    count_ = int(child_f.get('COUNT')) #count=1，0，代表扫描几次
    power_ = int(child_f.get('POWER')) #功率
    speed_ = int(child_f.get('SPEED')) #速度
    print (index_, type_, count_, power_, speed_)
    data_feature=pd.concat([data_feature,pd.DataFrame({'laser_index': [index_], 'type': [type_],
                                                       'count': [count_], 'power': [power_], 'speed': [speed_]})],
                           ignore_index = True)
    
#扫描策略排序
    type_i_line_point = []
    for i,child_p in enumerate(child_f.iterfind("POLYLINE")):
        polyline_array = np.array(child_p.text.split(',')).astype(float)
        type_i_line_point.append(list(polyline_array[1:]))
    num_poly.append(len(type_i_line_point))
    all_type_line_point.append(type_i_line_point)

data_feature['num_poly'] = num_poly   
data_feature['polyline'] = all_type_line_point

0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 215 1200
0 11 0 0 0
0 5 1 225 1200
0 11 0 0 0
0 5 1 235 1200
0 11 0 0 0
0 5 1 245 1200
0 11 0 0 0
0 5 1 255 1200
0 11 0 0 0
0 5 1 265 1200
0 11 0 0 0
0 5 1 275 1200
0 11 0 0 0
0 5 1 285 1200
0 11 0 0 0
0 5 1 295 1200


In [22]:
#绘制路径
def img_path(poly_data):
    '''路径绘图, 输入特征数据'''
    plt.figure(figsize=(15, 15))
    for i in range(len(poly_data)):
        poly_data[i] = np.array(poly_data[i]).reshape(-1,1,2)
        plt.plot(poly_data[i][:,:,0],poly_data[i][:,:,1],ls = '-',c='r')
        plt.xlim(0,258)
        plt.ylim(0,258)

In [23]:
## 按特征顺序逐特征保存路径轨迹图
def save_pathmap(data_feature):
    '''按特征顺序逐特征保存路径轨迹图'''
    for i in range(len(data_feature)):
        poly_data = data_feature.loc[i, 'polyline']
        type_ = data_feature.loc[i, "type"]
        img_path(poly_data)
        plt.title("type"+str(type_),size=20)
        plt.savefig("path_"+str(i)+"_type"+str(type_)+".jpg")

In [ ]:
##特征选择方式1
# att_type关注特征
def select_feature(data_feature,att_type=[]):
    if len(att_type)==0:
        data_feature_ = data_feature.loc[(data_feature["power"] != 0) & (data_feature["count"] != 0)] #提取激光扫描的特征
        _index = data_feature_.index.values #激光扫描的特征行索引        
    else:
        try:
            ##特征选择方式2---关注5，6，7特征
            data_feature_ = data_feature[data_feature.type.isin(att_type)]
            _index = data_feature_.index.values # 关注特征行索引
        except:
            print("特征不存在，重新输入")
            att_type=list(input("输入关注特征："))
            
    return data_feature_, _index

In [ ]:
#####功率信号分割#####
def power_index(data11,data_13_nonoise,gl_fg,data_feature_,cur_up_index, cur_down_index):
    GL_INDEX = []
    for i in range(len(data_13_nonoise) - 1):
        if abs(data_13_nonoise[i + 1] - data_13_nonoise[i]) >= gl_fg:
            GL_INDEX.append(i)# 功率变化索引
    GL_INDEX.append(0)
    GL_INDEX_S = []
    XH_INDEX=[]
    all_type_line_signal=[]
    all_type_line_signal_num=[]
    for i in range(len(GL_INDEX) - 1):#连续点筛选剔除
        if abs(GL_INDEX[i + 1] - GL_INDEX[i]) >= 2:
            GL_INDEX_S.append(GL_INDEX[i])
    for i in range(len(GL_INDEX_S)-1):#功率信号段索引
        XH_INDEX.append([GL_INDEX_S[i],GL_INDEX_S[i+1]])
    # print(XH_INDEX)
    for i in range(len(XH_INDEX)):
        '''
        解决功率上升下降沿与开关上升下降沿信号错位问题，但代码计算过于繁复，可考虑直接用dataframe的index处理，
        或用递归取第一个符合条件值；
        现解决方法：
        开关上升沿-功率上升沿，取最小非负值，作为特征信号起始索引；
        开关下降沿-功率下降沿，取最大非正值作为信号段结束索引
        '''
        SS_D = [] #上升
        XJ_D = [] #下降
        for j in range(len(cur_up_index)):
            SS_D.append(cur_up_index[j] - XH_INDEX[i][0])
            XJ_D.append(cur_down_index[j] - XH_INDEX[i][1])
        XH_INDEX[i][0] = cur_up_index[SS_D.index(min(ss for ss in SS_D if ss >= 0))]  # 向上取距离最近的上升沿
        XH_INDEX[i][1] = cur_down_index[XJ_D.index(max(xj for xj in XJ_D if xj <= 0))]  # 向下取距离最近的下降沿
        type_i_line_signal=data11[XH_INDEX[i][0]:XH_INDEX[i][1]]#分特征信号区域
        all_type_line_signal.append(type_i_line_signal)
    data_feature_['XH_INDEX'] = XH_INDEX
    data_feature_['signal'] = all_type_line_signal
    return data_feature_

In [ ]:
constant_pw_split = 50 #功率分割判断常量

power_change_index = []
for i in range(len(mp_df['data_pw']) - pw_interval):
    if mp_df['data_pw'].iloc[i+pw_interval] - mp_df['data_pw'].iloc[i] >= constant_pw_split:
        power_change_index.append(i+pw_interval)
    elif mp_df['data_pw'].iloc[i+pw_interval] - mp_df['data_pw'].iloc[i] <= -constant_pw_split:
        power_change_index.append(i)
# 阶跃大于constant_pw_split存在两个或以上连续点，剔除并留一个
power_change_index_new = [power_change_index[i] for i in range(len(power_change_index)-1)\
                         if abs(power_change_index[i+1]-power_change_index[i]) >=2] #保留一组最后点数,
power_change_index_new.append(power_change_index[-1])

In [ ]:
constant_sw_split = 2500

mp_df['data_sw'] = mp_df['data_sw'].apply(lambda x:0 if x <= constant_sw_split else 1)
indexup = mp_df[mp_df['data_sw'].diff()>0].index.to_numpy()
indexdown = mp_df[mp_df['data_sw'].diff()<0].index.to_numpy()

In [ ]:
##建立熔道和强度索引
##已知索引行--按照表单索引建立
def line_signal2(data_feature_att,_index,data11,cur_up_index, cur_down_index):
    '''
    data_feature——关注特征数据,dataframe
    _index——关注特征数据行索引
    data11——强度信号数据
    cur_up_index——上升沿索引
    cur_down_index——下降沿索引
    '''
    all_dx_line_signal=[]
    for i in _index:
        dx_line_signal = []
        type_i_att=data_feature_att.loc[i, "type"]
        type_i_line_point_att= data_feature_att.loc[i, 'polyline']
        type_i_line_XH_INDEX= data_feature_att.loc[i,'XH_INDEX'] #信号区间索引
        # print(type_i_line_point_att,type_i_att,type_i_line_XH_INDEX)
        dx_line_num=len(type_i_line_point_att) #该特征熔道数
        # 该特征开关信号索引区间
        begin_=cur_up_index.index(type_i_line_XH_INDEX[0])
        end_=cur_down_index.index(type_i_line_XH_INDEX[1])+1 ##起始是0，索引时+1
        dx_up_XH_INDEX = cur_up_index[begin_:end_]
        dx_down_XH_INDEX = cur_down_index[begin_:end_]
        
        i_dx_signal=[]
        for j in range(len(dx_up_XH_INDEX)):
            dx_signal = data11[dx_up_XH_INDEX[j]:dx_down_XH_INDEX[j]]  # 分割特征信号，用上升沿和下降沿取熔道信号段
            i_dx_signal.append(dx_signal) #单道信号索引[s1,s2,...]
        dx_signal_num=len(i_dx_signal) #该特征熔道信号数
        # print(i_dx_signal,dx_signal_num)
        ##判断信号与熔道维度
        '''目前该方法没有判断哪里缺失，直接前后舍弃或者前后补0'''
        if dx_line_num <= dx_signal_num: #分割信号多或者相等
            i_dx_signal=i_dx_signal[0:dx_line_num]
        else: #分割信号缺失，填充，补0，前后补0，保持维度一致
            i_dx_signal=i_dx_signal+[[0]*4]*(dx_line_num - dx_signal_num)
        #连接索引
        for j in range(dx_line_num):
            dx_line_signal.append([type_i_att,j,type_i_line_point_att[j],i_dx_signal[j]]) #单索引[type,熔道序号，熔道[x,y]，信号]
        data_feature_att.loc[i, 'dx_line_signal'] = [dx_line_signal]
    all_dx_line_signal.append(dx_line_signal)
    return data_feature_att, all_dx_line_signal

- 逐特征：
  - 信号区间内，计算特征熔道数
  - 创建特征开关信号索引区间，即开关上升沿和下降沿的索引区间
  - for循环，用上升沿和下降沿遍历分割取熔道信号段，形成单道信号组成的索引列表，计算特征下熔道信号数
  - 进行路径和信号数维度判断：
     - 如分割信号大于等于路径，信号索引向后舍弃；
     - 反之，向后补零
     - - 遗留问题：未能有效定位，如中间丢失数据呢？
     
  - 连接索引
     - 创建[type, 熔道序号， 熔道[x,y],信号段]组成的列表
     - 给关注特征data_feature_att，创建路径-信号关联列
     - - 问题：dataframe中再用列表，未能发挥dataframe的作用，dataframe本身自带索引功能，未能有效利用。

In [ ]:
##绘制关注熔道的轨迹和强度图
def dx_line_signal_map(_index, m=0, n=-1):
    for i in _index:
        line_signal = data_feature_att.loc[i, 'dx_line_signal'][0][m:n]
        line_signal.append(data_feature_att.loc[i, 'dx_line_signal'][0][n])
        # print(line_signal) #获取关注特征信息
        poly_line={}
        poly_signal=[]
        plt.figure(figsize=(20, 15))
        plt.subplot(2, 1, 1)
        for j in range(len(line_signal)):
            poly_line[j] = np.array(line_signal[j][2]).reshape(-1, 1, 2) #获取轨迹
            plt.plot(poly_line[j][:, :, 0], poly_line[j][:, :, 1], ls='-', c='r')
            plt.xlim(0, 250)
            plt.ylim(0, 250)
            poly_signal.append(line_signal[j][3]) #获取对应强度
        plt.title("index_"+str(i)+"_path",size=20)
        plt.subplot(2, 1, 2)
        mn_y =np.concatenate(poly_signal,axis=0)  #强度合并
        
        mn_x = range(len(mn_y))
        plt.plot(mn_x, mn_y, '-', c='g')
        plt.title("index_"+str(i)+"_signal",size=20)
        plt.savefig("index_"+str(i)+".jpg")
        # plt.show()